# Tables Total Recall

The purpose of this notebook is to get to the point where we can extract 100% of part names from transistor hardware sheets.

In [ ]:
%load_ext autoreload
%autoreload 2

In [4]:
load_pickle = False
save_pickle = False

corpus_loaded = False
if load_pickle:
    try:
        import cPickle
        with open("data/hardware/hardware_corpus.pkl","r") as pkl:
            %time corpus = cPickle.load(pkl)
        corpus_loaded = True
        print "Corpus has been loaded."
    except:
        print "Corpus could not be loaded."
        print "Corpus will be parsed instead..."
if not corpus_loaded:
    from snorkel.parser import CorpusParser
    from snorkel.parser import HTMLOmniParser
    from snorkel.parser import SentenceParser

    doc_parser = HTMLOmniParser(path='data/hardware/hardware_html/')
    context_parser = SentenceParser()
    #     table_parser = TableParser()

    cp = CorpusParser(doc_parser, context_parser, max_docs=15)
    %time corpus = cp.parse_corpus(name='Hardware Corpus')
    print "Corpus has been parsed."
    
    if save_pickle:
        with open("data/hardware/hardware_corpus.pkl","w") as pkl:
            %time cPickle.dump(corpus, pkl)
            print "Corpus has been pickled."

CPU times: user 2.92 s, sys: 72.7 ms, total: 2.99 s
Wall time: 20 s
Corpus has been parsed.


In [2]:
from load_dictionaries import load_hardware_dictionary

gold_parts = load_hardware_dictionary()
print "Loaded %s gold part numbers." % len(gold_parts)

Loaded 179 gold part numbers.


In [8]:
from snorkel.candidates import Ngrams, EntityExtractor
from snorkel.matchers import RegexMatchEach, DictionaryMatch, RangeMatcher

ngrams = Ngrams(n_max=2)
part_matcher = DictionaryMatch(d=gold_parts, longest_match_only=False)
part_extractor = EntityExtractor(ngrams, part_matcher)

%time parts = part_extractor.extract(corpus.get_sentences(), name='all')
for p in parts[:5]: 
    print p
print "Extracted %s candidate part numbers." % len(parts)

CPU times: user 1.43 s, sys: 42.1 ms, total: 1.47 s
Wall time: 1.44 s
Span("BC546", context=None, chars=[0,4], words=[0,0])
Span("BC546B", context=None, chars=[188,193], words=[41,41])
Span("BC547A", context=None, chars=[196,201], words=[43,43])
Span("BC548B", context=None, chars=[210,215], words=[49,49])
Span("BC548", context=None, chars=[512,516], words=[95,95])
Extracted 68 candidate part numbers.


In [9]:
from fractions import Fraction
g = set(gold_parts)
x = set([p.get_span() for p in parts])
recall = Fraction(len(g.intersection(x)),len(g))
precision = Fraction(len(g.intersection(x)),len(x))
print "Recall: %0.3f (%s/%s)" % (float(recall), recall.numerator, recall.denominator)
print "Precision: %0.3f (%s/%s)" % (float(precision), precision.numerator, precision.denominator)

Recall: 0.112 (20/179)
Precision: 0.952 (20/21)


In [11]:
print x - g
for p in x:
    print p

set([u'2n4123'])
BC327
2n4123
BC338
BC337-25
2N4124
BC548B
BC337
2N3906
BC546B
BC547A
2N4123
MMBT3906
PZT3906
BC182
BC549
BC548
2N6426
BC550
MMBT6427
BC547
BC546


In [ ]:
# ranges (e.g., BC546-BC548) or lists (e.g., BC546/547/548)